<a href="https://colab.research.google.com/github/kaitz2981-ux/TTNT-Tuan03/blob/main/Tuan3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

    import math

# Định nghĩa các hằng số
    PLAYER_X = 'X'  # AI (Maximizer)
    PLAYER_O = 'O'  # Người chơi (Minimizer)
    EMPTY = ' '
    BOARD_SIZE = 3

    class TicTacToeAI:
    def __init__(self):
# Khởi tạo bảng 3x3 trống
    self.board = [[EMPTY for _ in range(BOARD_SIZE)] for _ in range(BOARD_SIZE)]

    def _check_win(self, player):
# 1. Kiểm tra hàng
    for row in range(BOARD_SIZE):
    if all(self.board[row][col] == player for col in range(BOARD_SIZE)):
    return True
        
# 2. Kiểm tra cột
    for col in range(BOARD_SIZE):
    if all(self.board[row][col] == player for row in range(BOARD_SIZE)):
    return True
        
# 3. Kiểm tra đường chéo chính
    if all(self.board[i][i] == player for i in range(BOARD_SIZE)):
    return True
    
# 4. Kiểm tra đường chéo phụ
    if all(self.board[i][BOARD_SIZE - 1 - i] == player for i in range(BOARD_SIZE)):
    return True
           
    return False

    def _is_moves_left(self):
# Kiểm tra xem còn ô trống nào trên bảng không.
    for row in range(BOARD_SIZE):
    for col in range(BOARD_SIZE):
    if self.board[row][col] == EMPTY:
    return True
    return False

    def evaluate(self):
# Trả về +10 nếu AI (X) thắng.
    if self._check_win(PLAYER_X):
    return 10
# Trả về -10 nếu Người chơi (O) thắng.
    if self._check_win(PLAYER_O):
    return -10
# Trả về 0 nếu hòa hoặc chưa kết thúc
    return 0

    def minimax(self, depth, is_maximizing_player):
# 1. TRƯỜNG HỢP CƠ SỞ (Terminal Node)
    score = self.evaluate()
# AI (X) thắng
    if score == 10:  
# Ưu tiên thắng sớm
    return score - depth  
# Người chơi (O) thắng
    if score == -10:
# Ưu tiên thua muộn
    return score + depth  
# Hòa
    if not self._is_moves_left():
    return 0

# 2. LƯỢT CỦA MAXIMIZING PLAYER (AI - X)
    if is_maximizing_player:
    max_eval = -math.inf
            
    for row in range(BOARD_SIZE):
    for col in range(BOARD_SIZE):
    if self.board[row][col] == EMPTY:
# Thực hiện nước đi
    self.board[row][col] = PLAYER_X
                        
# Đệ quy và tìm giá trị lớn nhất
    eval = self.minimax(depth + 1, False)
    max_eval = max(max_eval, eval)
                        
# Hoàn tác nước đi (Backtrack)
    self.board[row][col] = EMPTY
                        
    return max_eval

# 3. LƯỢT CỦA MINIMIZING PLAYER (Người chơi - O)
    else:
    min_eval = math.inf
            
    for row in range(BOARD_SIZE):
    for col in range(BOARD_SIZE):
    if self.board[row][col] == EMPTY:
# Thực hiện nước đi
    self.board[row][col] = PLAYER_O
                        
# Đệ quy và tìm giá trị nhỏ nhất
    eval = self.minimax(depth + 1, True)
    min_eval = min(min_eval, eval)
                        
# Hoàn tác nước đi (Backtrack)
    self.board[row][col] = EMPTY
                        
    eturn min_eval

    def find_best_move(self):
# Tìm nước đi tối ưu nhất cho AI (X)
    best_move_value = -math.inf
    best_move = (-1, -1)
        
# Duyệt qua tất cả các ô trống
    for row in range(BOARD_SIZE):
    for col in range(BOARD_SIZE):
    if self.board[row][col] == EMPTY:
# 1. Thực hiện nước đi
    self.board[row][col] = PLAYER_X
                    
# 2. Gọi Minimax (lượt tiếp theo là của Min)
    move_value = self.minimax(0, False)
                    
# 3. Hoàn tác
    self.board[row][col] = EMPTY
                    
# 4. Cập nhật nước đi tốt nhất
    if move_value > best_move_value:
    best_move_value = move_value
    best_move = (row, col)
                        
    print(f"Giá trị tốt nhất tìm được: {best_move_value}")
    return best_move

# --- Hàm chạy thử nghiệm ---
    def print_board(board):
# In bảng để dễ theo dõi
    print("-----")
    for row in board:
    print("|" + "|".join(row) + "|")
    print("-----")

    def test_minimax():
    game_ai = TicTacToeAI()
    
# Thiết lập một trạng thái ví dụ (AI cần thắng trong 1 nước)
    game_ai.board = [
        ['X', 'X', EMPTY],
        ['O', EMPTY, EMPTY],
        [EMPTY, EMPTY, EMPTY]
    ]
    
    print("Trạng thái bảng ban đầu:")
    print_board(game_ai.board)
    
    best_move_row, best_move_col = game_ai.find_best_move()
    
    print(f"\nNước đi tối ưu nhất cho AI (X) là: ({best_move_row}, {best_move_col})")
    
# Thực hiện nước đi tối ưu
    if best_move_row != -1:
    game_ai.board[best_move_row][best_move_col] = PLAYER_X
        
    print("\nTrạng thái bảng sau khi AI đi:")
    print_board(game_ai.board)
    
    if game_ai._check_win(PLAYER_X):
    print("AI (X) đã thắng!")

# Chạy thử nghiệm
# test_minimax()